## Crops atlas and saves as mhd file for image registration
### Do not save any segmentation files in imageJ, imageJ some how mess up several ids with very high values!

In [24]:
from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache

import os
import re

import numpy as np

import SimpleITK as sitk

import tkinter.filedialog as fdialog

import Neuron_analysis as na
from Neuron_analysis import *

In [5]:
outdir = fdialog.askdirectory(title='Please select the output directory')

In [25]:
# Get segmentation file from allenSDK

mcc = MouseConnectivityCache(resolution=25)

annot, annot_info = mcc.get_annotation_volume()


In [4]:
def get_template_name(outdir):
    '''Get corresponding template name for finding the coronal crop'''
    files=os.listdir(outdir)
    template_name=[i for i in files if re.search("template.+\.mhd",i)][0]
    return template_name

In [11]:
template_name= get_template_name(outdir)
lead,trail= na.find_crop(template_name)
# identify cropped section

end= 528 -trail

lead, end

(92, 443)

In [26]:
import napari
viewer = napari.Viewer()
viewer.add_labels(annot, name='atlas')

C:\Users\liu\AppData\Local\Continuum\anaconda3\lib\site-packages\napari\_vispy\vispy_camera.py:109: RuntimeWarning: divide by zero encountered in true_divide
  zoom = np.min(canvas_size / scale)


<Labels layer 'atlas' at 0x27d82363ef0>

In [27]:
def create_mask(x):
    #create mask based on selected atlas index (an int)
    mask=np.zeros_like(annot)
    mask_idx= annot==x
    mask[mask_idx]=1
    return mask

In [28]:
img_id=312782592
mask=create_mask(img_id)
viewer.add_labels(mask, name=f'{img_id}')

<Labels layer '312782592' at 0x27d99a918d0>

In [38]:
#structure_tree.get_structures_by_id([312782566])

[{'acronym': 'VISa6a',
  'graph_id': 1,
  'graph_order': 351,
  'id': 312782566,
  'name': 'Anterior area, layer 6a',
  'structure_id_path': [997, 8, 567, 688, 695, 315, 22, 312782546, 312782566],
  'structure_set_ids': [667481449, 184527634, 396673091, 691663206],
  'rgb_triplet': [0, 159, 172]}]

In [20]:
crop_annot =annot [lead:end,:,:]
image = sitk.GetImageFromArray(crop_annot)

In [21]:
outputImageFileName= f'/atlas_{lead}-{end}.mhd'

writer = sitk.ImageFileWriter()
writer.SetFileName(outdir + outputImageFileName)
writer.Execute(image)

'D:/2PT electroporation time/GF243atlas_92-443.mhd'

In [46]:
# reader = sitk.ImageFileReader()
# reader.SetFileName("test_atlas.mhd")
# image = reader.Execute();

C:\Users\liu\AppData\Local\Continuum\anaconda3\lib\site-packages\napari\_vispy\vispy_camera.py:109: RuntimeWarning: divide by zero encountered in true_divide
  zoom = np.min(canvas_size / scale)


<Labels layer 'atlas' at 0x132ba147b38>

(528, 320, 456)

(351, 320, 456)

In [30]:
atlas_name='D:/2PT electroporation time/GF243/ara2sample_atlas/result.mhd'
atlas2= sitk.ReadImage(atlas_name) 
atlas2= sitk.GetArrayFromImage(atlas2)

In [32]:
viewer.add_labels(atlas2, name='atlas2')

C:\Users\liu\AppData\Local\Continuum\anaconda3\lib\site-packages\napari\layers\labels\labels.py:839: RuntimeWarning: overflow encountered in long_scalars
  if max_label_val * nbytes_low_discrepancy > max_nbytes:
C:\Users\liu\AppData\Local\Continuum\anaconda3\lib\site-packages\napari\layers\labels\labels.py:839: RuntimeWarning: overflow encountered in long_scalars
  if max_label_val * nbytes_low_discrepancy > max_nbytes:


<Labels layer 'atlas2' at 0x27d9545c4a8>

In [1]:
from allensdk.core.reference_space_cache import ReferenceSpaceCache

reference_space_key = 'annotation/ccf_2017'
resolution = 25
rspc = ReferenceSpaceCache(resolution, reference_space_key, manifest='manifest.json')
# ID 1 is the adult mouse structure graph
tree = rspc.get_structure_tree(structure_graph_id=1) 

In [4]:
rsp = rspc.get_reference_space()
rsp.write_itksnap_labels('ccf_2017_itksnap.nrrd', 'ccf_2017_itksnap_labels.txt')

2021-09-23 17:54:29,962 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://download.alleninstitute.org/informatics-archive/current-release/mouse_ccf/annotation/ccf_2017/annotation_25.nrrd


In [5]:
rsp.write_itksnap_labels?

Signature: rsp.write_itksnap_labels(annotation_path, label_path, **kwargs)
Docstring:
Generate a label file (nrrd) and a label_description file (csv) for use with ITKSnap

Parameters
----------
annotation_path : str
    write generated label file here
label_path : str
    write generated label_description file here
**kwargs : 
    will be passed to self.export_itksnap_labels
File:      c:\users\liu\appdata\local\continuum\anaconda3\lib\site-packages\allensdk\core\reference_space.py
Type:      method
